In [1]:
import cv2
import numpy as np
import csv
from datetime import datetime
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mtcnn = MTCNN(keep_all=True, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def add_new_user(frame, name):
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])
    boxes, _ = mtcnn.detect(rgb_small_frame)

    if boxes is not None and len(boxes) > 0:
        boxes = boxes[:, :4].astype(int)
        face = rgb_small_frame[boxes[0][1]:boxes[0][3], boxes[0][0]:boxes[0][2], :]
        face = cv2.resize(face, (160, 160))  
        face = torch.tensor(face, dtype=torch.float).to(device).permute(2, 0, 1).unsqueeze(0)
        with torch.no_grad():
            face_encode = model(face)[0].detach().cpu().numpy()
            
        face_encode = np.squeeze(face_encode)
        print(f"Shape of face_encode: {face_encode.shape}")

        known_face_encoding.append(face_encode)
        known_faces_names.append(name)
        students.append(name)

known_face_encoding = []
known_faces_names = []
students = []

try:
    with open('known_face.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            known_faces_names.append(row[0])
            known_face_encoding.append(np.fromstring(row[1], dtype=float, sep=',').reshape(-1))
            students.append(row[0])
except FileNotFoundError:
    pass

now = datetime.now()
current_date = now.strftime("%Y-%d")
f = open(current_date + '.csv', 'w+', newline='')
lnwriter = csv.writer(f)
column_names = ["Name", "Time"]
lnwriter.writerow(column_names)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])
    face_locs, _ = mtcnn.detect(rgb_small_frame)
    face_encodes = []
    for loc in face_locs:
        y1, x1, y2, x2 = [int(coord) for coord in loc]
        face = rgb_small_frame[y1:y2, x1:x2, :]
        face = cv2.resize(face, (160, 160)) 
        face = torch.tensor(face, dtype=torch.float).to(device).permute(2, 0, 1).unsqueeze(0)
        face_encodes.append(model(face).detach().cpu().numpy())

    face_names = []

    if known_face_encoding:
        for face_encode in face_encodes:
            matches = [np.linalg.norm(known_encode - face_encode) < 0.6 for known_encode in known_face_encoding if known_encode.shape[0] > 0]
            if any(matches):
                first_match_index = matches.index(True)
                name = known_faces_names[first_match_index]
            else:
                name = "Unknown"
            face_names.append(name)
    
            if name != "Unknown":
                if name in known_faces_names:
                    if name in students:
                        students.remove(name)
                        current_time = now.strftime("%H-%M-%S")
                        lnwriter.writerow([name, current_time])
    else:
        print("No known face encodings available.")

    cv2.imshow("Attendance", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('a'):
        user_name = input("Enter the new user's name: ")
        add_new_user(frame, user_name)

with open('known_face.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for name, encoding in zip(known_faces_names, known_face_encoding):
        encoding_str = ",".join(map(str, encoding))
        csv_writer.writerow([name, encoding_str])

cap.release()
cv2.destroyAllWindows()
f.close()